# Patient Reviewer

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from JupyterReviewer.Reviewers.PatientReviewer import PatientReviewer
from JupyterReviewer.ReviewData import ReviewDataAnnotation
from JupyterReviewer.ReviewDataApp import AppComponent

import pandas as pd
from dash import html

## Edit the code below according to the commented directions

In [17]:
reviewer = PatientReviewer()

reviewer.set_review_data(
    # Enter the file path where you would like your pkl file of your review data to be stored
    review_data_fn='/Users/svanseters/Broad/JupyterReviewer/Reviewer.pkl',
    # Breifly describe the source of your data and its purpose
    description='first review',
    # Enter the file path to your data to be reviewed
    df=pd.read_csv(f'/Users/svanseters/Broad/JupyterReviewer/example_notebooks/example_data/updated_test_data_for_reviewer.csv').set_index('participant_id'),
    preprocess_data_dir = '/Users/svanseters/Broad/JupyterReviewer/preprocess_data',
    samples_fn='/Users/svanseters/Broad/JupyterReviewer/samples.csv'
)

reviewer.set_review_app(biospecimens_fn='/Users/svanseters/Broad/JupyterReviewer/biospecimens.7oqli.tsv', samples_fn='/Users/svanseters/Broad/JupyterReviewer/samples.csv', preprocess_data_dir = '/Users/svanseters/Broad/JupyterReviewer/preprocess_data', drivers_fn='drivers.csv')


Loading existing review session /Users/svanseters/Broad/JupyterReviewer/Reviewer.pkl...


### Pass custom column color assignments into the review app with the following syntax:

In [35]:
# reviewer.set_review_app(custom_colors=[
#     [column_id_1, filter_query_1, text_color_1, background_color_1], 
#     [<same for all additional columns/queries>]
# ])

#Example (comment out reviewer.set_review_app() above and uncomment below to try):

# reviewer.set_review_app(custom_colors=[
#     ['Variant_Classification', 'Silent', 'Black', 'LightBlue'], 
#     ['Variant_Classification', 'Missense_Mutation', 'White', 'Indigo']
# ])

### Add your own components below 

In [7]:
# reviewer.app.add_component(AppComponent('component name', layout)))

### Add your own annotations below

In [8]:
# reviewer.review_data.add_annotation('annot name', ReviewDataAnnotation())

## Run

In [18]:
reviewer.run(mode='external', port=8086)

Dash app running on http://0.0.0.0:8086/


/Users/svanseters/Broad/JupyterReviewer/reviewEnv/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/

/Users/svanseters/Broad/JupyterReviewer/JupyterReviewer/AppComponents/CNVPlotComponent.py:261: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/svanseters/Broad/JupyterReviewer/JupyterReviewer/AppComponents/CNVPlotComp